# Detect diabetic retinopathy to stop blindness before it's too late

we'll build a machine learning model to speed up disease detection. we’ll work with thousands of images collected in rural areas to help identify diabetic retinopathy automatically. If successful, it will not only help to prevent lifelong blindness, but these models may be used to detect other sorts of diseases in the future, like glaucoma and macular degeneration.

The dataset contains around 3.5ktraining images of retina which have been labelled by clinicians for the severity of diabetic retinopathy on a scale of 0 to 4:

0 - No DR

1 - Mild

2 - Moderate

3 - Severe

4 - Proliferative DR

We will use Fastai to solve this problem. 
## What is Fastai

The fastai library simplifies training fast and accurate neural nets using modern best practices. It's based on research in to deep learning best practices undertaken at fast.ai, including "out of the box" support for vision, text, tabular, and collab (collaborative filtering) models.

### Let's get started with the code.

The following 3 lines ensure that any edits to libraries you make are reloaded here automatically, and also that any charts or images displayed are shown in this notebook.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

We import all the necessary packages. We are going to work with the fastai V1 library which sits on top of Pytorch 1.0. The fastai library provides many useful functions that enable us to quickly and easily build neural networks and train our models.

In [ ]:
from fastai import *
from fastai.vision import *
import pandas as pd
import matplotlib.pyplot as plt

If you're using a computer with an unusually small GPU, you may get an out of memory error when running this notebook. If this happens, click Kernel->Restart, uncomment the 2nd line below to use a smaller batch size and try again.

In [ ]:
bs = 64
# bs = 16   # uncomment this line if you run out of memory even after clicking Kernel->Restart

# Looking at the data

Below, We have defined a path variable to navigate to the data source for the problem.

We have created 2 different variables for train csv file (which contains image file names and their respective labels) and to the image data source folder itself which contains all the images. (train_img_path).

In [ ]:
import os
path = '../input/aptos2019-blindness-detection/'
train_csv_path = path +'/train.csv'
train_img_path = path + 'train_images/'
train = pd.read_csv(train_csv_path)

#test path strings
print(train_csv_path)
print(train_img_path)

In [ ]:
train.head()

In [ ]:
print("There are total {} images in training dataset".format(len(train)))

In [ ]:
f_names = get_image_files(train_img_path)
f_names[:5]

Fastai ImageList.from_folder() enables access to fetch image files from a folder (imagenet style) using single line of code.

Let's take a look at one of the image files using Imagelist.

In [ ]:
il = ImageList.from_folder(train_img_path)
il

In [ ]:
il.open(il.items[10])

In [ ]:
train['id_code'] = train['id_code'].map(lambda x: (train_img_path + x + '.png'))

#test the paths
print(train['id_code'][1])
print(train['id_code'][2])

The below code transforms the images by flipping them horizontally and vertically along with rotating, zooming, lighting etc to make the model more robust by boosting the image samples available for training.

In [ ]:
tfms = get_transforms(do_flip=True,flip_vert=True,
                      max_rotate=360,max_warp=0,max_zoom=1.1,
                      max_lighting=0.1,p_lighting=0.5)

Before any work can be done a dataset needs to be converted into a DataBunch object, and in the case of the computer vision data - specifically into an ImageDataBunch subclass.

The from_df() method allows to conviniently fetch data from a dataframe and create a dataBunch object. The ImageDataBunch object contains both training set and validation set and a optional test set. 

The ImageDataBunch helps setup the model training process very quickly.

In the below code, we have notified the ImageDataBunch object that our data is in the "train" dataframe and our label column is "diagnosis". 

Also, the image sizes are currently non coherent (of different sizes) in our dataset. It is very important to regularize the images to the same size before training. 

Also we have set the batchsize to 64. This simply means that we are showing 64 images to the model at once during training.

In [ ]:
data = ImageDataBunch.from_df(path = '', df= train, label_col='diagnosis', ds_tfms=tfms,
                              valid_pct=0.2, size=224, bs=64).normalize(imagenet_stats)

Show_batch() is a handy method of ImageDataBunch which displays sample images from our dataset.

In [ ]:
data.show_batch(rows = 4, figsize= (12,10))

In [ ]:
print(data.classes)

The evaluation metric for the competition is quadratic kappa score. The good folk's at FASTAI have already implemented this metric.

In a nutshell, this metric determines the agreement between predicted labels and ground truth.
e.g. If the prediction of a particular sample is mild but the ground truth is severe, the metric penalizes more compared to if the prediction is mild and ground truth is moderate.

In [ ]:
kappa = KappaScore()
kappa.weights = "quadratic"

Once we have our data in the dataBunch object  specifically ImageDataBunch for Computer vision related problems), we are ready to kickstart our training process. We can choose from a number of model architectures available in Fastai model Zoo.

Here, we use Resnet34 and Resnet50 which arguably work pretty well for the problem at hand.

# Training: resnet34

Now we will start training our model. We will use a convolutional neural network backbone and a fully connected head with a single hidden layer as a classifier.

We will train for 4 epochs (4 cycles through all our data).

Resnet34, as the name indicates is a 34 layer deep neural network which has been pretrained on Imagenet.

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=[error_rate, kappa])

Let's see below how the neural network's architecture looks like:

In [ ]:
learn.model

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('stage-1')

Let's take a look at some images where our model went wrong. We see what the model predicted, what was the ground truth and what was the loss.

In [ ]:
interpret = ClassificationInterpretation.from_learner(learn)
losses,idx = interpret.top_losses()

interpret.plot_top_losses(9, figsize=(15,11))

In [ ]:
interpret.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interpret.most_confused(min_val=2)

# Unfreezing, finetuning and learning rates

#### Since our model is working as we expect it to, we will unfreeze our model and train some more.

### Explaining the unfreezing step:

In the steps above, **we just trained the top few layers of the neural net**. Now, we will unfreeze the entire neural net and train it with the learning rate found with lr_find(). Explanations below:

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(1)

Let's load the previous state of the model.

In [ ]:
learn.load('stage-1')

It is known that the learning rate is the most important hyper-parameter to tune for training deep neural networks. A new method for setting the learning rate named cyclical learning rates, which practically eliminates the need to experimentally find the best values and schedule for the global learning rates. Instead of monotonically decreasing the learning rate, this method lets the learning rate cyclically vary between reasonable boundary values. Training with cyclical learning rates instead of fixed values achieves improved classification accuracy without a need to tune and often in fewer iterations. [link here](https://arxiv.org/abs/1506.01186)

lr_find() does this for us. 

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

The above plot shows us the optimal learning rates to train the nn. So now Let's unfreeze the entire model and train it with the learning rates determined from the plot above. We train the model for 2 epochs.

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=slice(1e-06,1e-03))

That's a fairly accurate model. Now let's jump to resnet50 and see how it performs better than resnet34.

# Training: resnet50
Now we will train in the same way as before but with one caveat: instead of using resnet34 as our backbone we will use resnet50 (resnet34 is a 34 layer residual network while resnet50 has 50 layers.

Basically, resnet50 usually performs better because it is a deeper network with more parameters. Let's see if we can achieve a higher performance here. To help it along, let's us use larger images too, since that way the network can see more detail. We reduce the batch size a bit since otherwise this larger network will require more GPU memory.

In [ ]:
data = ImageDataBunch.from_df(path = '', df= train, label_col='diagnosis', ds_tfms=tfms,
                              valid_pct=0.2, size=299, bs=32).normalize(imagenet_stats)

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=[error_rate,kappa])

In [ ]:
learn.lr_find()
learn.recorder.plot()

### Feel free to fork the kernel and Uncomment the below lines to train the Resnet50 model and see high it performs better than the above model.

We train the nn with resnet50 model architecture for 8 epochs.

In [ ]:
# learn.fit_one_cycle(8)

In [ ]:
# learn.save('stage-1-50')

Unfreeze all the layers and train again for 3 epochs and learning rates determined with lr_find()

In [ ]:
# learn.unfreeze()
# learn.fit_one_cycle(3, max_lr=(slice(1e-6,1e-4)))

In [ ]:
#If it doesn't, you can always go back to your previous model.
# learn.load('stage-1-50');

interpret the model performance intuitively:

In [ ]:
# interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
# interp.most_confused(min_val=2)

This tutorial is taken from the Fastai V3 MOOC taught by the legendary **Jeremy Howard** who is also one of the founder of Fastai.

Further reading : https://course.fast.ai/

**Please drop an upvote if you find this helpful :)**